In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 100.3 MB/s eta 0:00:00


In [1]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

In [2]:
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-east4"
vertexai.init(project=PROJECT_ID, location=REGION)

# Load a generative model

In [3]:
model = GenerativeModel("gemini-pro")

In [4]:
creative_gen_config = GenerationConfig(temperature=1, top_p=0.8)

In [5]:
prompt = "What are some strategies for overcoming writer's block?"

response = model.generate_content(prompt,
                                  generation_config=creative_gen_config)
print(response.text)

## Strategies for Overcoming Writer's Block:

**1. Take a Break:**

* **Rest your mind:** Sometimes the best way to get your creative juices flowing again is to simply step away from your work. Take a walk, listen to music, or engage in another activity that allows you to clear your head.
* **Change your environment:** If your current workspace isn't inspiring you, try working in a different location, like a coffee shop, library, or park.
* **Do something physical:** Get your blood flowing with a quick workout or some stretches. Physical activity can help to boost your energy and creativity.

**2. Prime the Pump:**

* **Free writing:** Set a timer for 5 minutes and write whatever comes to mind, even if it's nonsensical. This can help you to overcome the initial hurdle of starting to write.
* **Brainstorm:** Use mind mapping or another brainstorming technique to generate new ideas.
* **Read:** Immerse yourself in the genre you're writing in or read about topics related to your subject m

# Text Classification

In [6]:
predictable_gen_config = GenerationConfig(temperature=0.1, top_p=0.1)

In [7]:
prompt = """
Given a user's input, classify their intent, such as "finding information", "making a reservation", or "placing an order". \n
user input: Hi, can you please book a table for two at Juan for May 1?
"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

## Intent: Making a reservation

The user is requesting to book a table for two people at a restaurant called Juan for May 1st. This falls under the category of "making a reservation". 



In [8]:
prompt = """
What is the topic for a given news headline? \n
- business \n
- entertainment \n
- health \n
- sports \n
- technology \n\n

Text: Pixel 7 Pro Expert Hands On Review. \n
The answer is: technology \n

Text: Quit smoking? \n
The answer is: health \n

Text: Birdies or bogeys? Top 5 tips to hit under par \n
The answer is: sports \n

Text: Relief from local minimum-wage hike looking more remote \n
The answer is: business \n

Text: You won't guess who just arrived in Bari, Italy for the movie premiere. \n
The answer is:
"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

entertainment 



In [9]:
import pandas as pd
from sklearn.metrics import classification_report

review_data_df = pd.DataFrame(
    {
        "review": [
            "i love this product. it does have everything i am looking for!",
            "all i can say is that you will be happy after buying this product",
            "its way too expensive and not worth the price",
            "i am feeling okay. its neither good nor too bad.",
        ],
        "sentiment_groundtruth": ["positive", "positive", "negative", "neutral"],
    }
)

review_data_df

,review,sentiment_groundtruth
0,i love this product. it does have everything i...,positive
1,all i can say is that you will be happy after ...,positive
2,its way too expensive and not worth the price,negative
3,i am feeling okay. its neither good nor too bad.,neutral


In [10]:
def get_sentiment(row):
    prompt = f"""Classify the sentiment of the following review as "positive", "neutral" or "negative". Return only the classification.
                review: {row}
              """
    response = model.generate_content(
        contents=prompt, generation_config=predictable_gen_config
    ).text
    return response


review_data_df["sentiment_prediction"] = review_data_df["review"].apply(get_sentiment)
review_data_df

,review,sentiment_groundtruth,sentiment_prediction
0,i love this product. it does have everything i...,positive,positive
1,all i can say is that you will be happy after ...,positive,"""positive"" \n"
2,its way too expensive and not worth the price,negative,negative
3,i am feeling okay. its neither good nor too bad.,neutral,neutral


In [11]:
report = classification_report(
    review_data_df["sentiment_groundtruth"], review_data_df["sentiment_prediction"]
)
Markdown(report)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

"positive" 
       0.00      0.00      0.00         0
    negative       1.00      1.00      1.00         1
     neutral       1.00      1.00      1.00         1
    positive       1.00      0.50      0.67         2

    accuracy                           0.75         4
   macro avg       0.75      0.62      0.67         4
weighted avg       1.00      0.75      0.83         4


# Text Summarization

In [12]:
predictable_gen_config = GenerationConfig(temperature=0.1, top_p=0.1)

In [13]:
prompt = """
Provide a very short summary, no more than three sentences, for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a "logical qubit," and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Summary:

"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

## Summary:

Quantum computers are sensitive to errors, which can be mitigated by using quantum error correction. This technique encodes information across multiple qubits, creating a "logical qubit" with lower error rates. By using logical qubits, quantum computers can achieve the accuracy needed for useful calculations. 



In [14]:
prompt = """
Provide a very short summary in four bullet points for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a "logical qubit," and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Bullet points:

"""

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
Markdown(response.text)

* Quantum computers use qubits, which are sensitive to errors.
* Current error rates are too high for useful applications.
* Quantum error correction is needed to reduce error rates.
* This involves encoding information across multiple qubits to form a "logical qubit". 


In [15]:
prompt = """
Please generate a summary of the following conversation and at the end summarize the to-do's for the support Agent:

Customer: Hi, I'm Larry, and I received the wrong item.

Support Agent: Hi, Larry. How would you like to see this resolved?

Customer: That's alright. I want to return the item and get a refund, please.

Support Agent: Of course. I can process the refund for you now. Can I have your order number, please?

Customer: It's [ORDER NUMBER].

Support Agent: Thank you. I've processed the refund, and you will receive your money back within 14 days.

Customer: Thank you very much.

Support Agent: You're welcome, Larry. Have a good day!

Summary:
"""

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
Markdown(response.text)

## Summary of Conversation:

* **Customer:** Larry received the wrong item.
* **Desired Resolution:** Larry wants to return the item and receive a refund.
* **Order Number:** [ORDER NUMBER]
* **Resolution:** The Support Agent processed the refund. 
* **Refund Timeline:** Larry will receive the refund within 14 days.

## To-Dos for Support Agent:

* **Confirm:** Verify with Larry that the refund amount and timeline are correct.
* **Provide Instructions:** Inform Larry about the return process (e.g., return label, shipping instructions).
* **Follow Up:** Check with Larry in a few days to ensure he received the refund and confirm his satisfaction. 


# Text Extraction

In [16]:
order = "We need eight grilled cheese sandwiches. Two with swiss cheese, three with muenster, three with cheddar."

prompt = """
    Break the customer's order into individual items with keys for the following fields:
    - item_name
    - cheese_selection

    Order:
    {order_field}
""".format(order_field=order)

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

```json
[
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Swiss"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Swiss"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  }
]
```


# Question Answering

In [17]:
prompt = """Q: Who was President of the United States in 1955?
              Which party did he belong to?
            A:
        """
response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

Dwight D. Eisenhower was President of the United States in 1955. He belonged to the Republican Party. 


In [18]:
context = """
Storage and content policy \n
How durable is my data in Cloud Storage? \n
Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
"""

question = "How is high availability achieved?"

prompt = f"""Answer the question given in the contex below:
Context: {context}?\n
Question: {question} \n
Answer:
"""

print("[Prompt]")
print(prompt)

print("[Response]")
response = model.generate_content(contents=prompt, generation_config=predictable_gen_config)
print(response.text)

[Prompt]
Answer the question given in the contex below:
Context: 
Storage and content policy 

How durable is my data in Cloud Storage? 

Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
?

Question: How is high availability achieved? 

Answer:

[R